In [1]:
# !pip uninstall -y numpy==1.23.1
# !pip install numpy==1.21.6
# !pip install --upgrade numpy
!pwd

/home/apsisdev/mobassir/data/tts/bn_GlowTTS


In [2]:
!nvidia-smi
import torch
import sys
sys.path.append('/home/apsisdev/mobassir/data/tts/bn_GlowTTS/TTS')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from trainer import Trainer, TrainerArgs

# GlowTTSConfig: all model related values for training, validating and testing.

from TTS.tts.utils.text.tokenizer import TTSTokenizer
# BaseDatasetConfig: defines name, formatter and path of the dataset.


from TTS.tts.configs.shared_configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig
#from TTS.configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig#GlowTTSConfig


from TTS.config import load_config


from trainer import Trainer, TrainerArgs

from TTS.utils.audio import AudioProcessor
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.configs.glow_tts_config import GlowTTSConfig

Mon Oct 31 20:42:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   39C    P8    26W / 370W |    216MiB / 24576MiB |     28%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# ## Install Coqui TTS
# ! pip install -U pip
# ! pip install TTS

# !git clone https://github.com/coqui-ai/TTS
# os.chdir('TTS/')
# !python3 -m pip install -r requirements.txt
# !python3 setup.py develop

is_pretrain = False

!ls

run-September-30-2022_10+55AM-0000000  train_bn_Glow_TTS_male.ipynb  TTS


In [4]:
# # !apt install -y espeak
# !apt install -y espeak-ng
!pwd

/home/apsisdev/mobassir/data/tts/bn_GlowTTS


In [5]:

# output_path = "tts_train_dir"
# if not os.path.exists(output_path):
#     os.makedirs(output_path)

male = True

meta_file = '/home/apsisdev/mobassir/data/tts/bn_tts/male/mono/metadata_female.txt'
root_path = '/home/apsisdev/mobassir/data/tts/bn_tts/male/mono'

def formatter(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the LJSpeech meta data file to TTS format
    https://keithito.com/LJ-Speech-Dataset/"""
    txt_file = meta_file
    items = []
    speaker_name = "ljspeech"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wav", cols[0] + ".wav")
            try:
                text = cols[1]
            except:
                print("not found")

            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


dataset_config = BaseDatasetConfig(
    dataset_name="ljspeech", meta_file_train="/home/apsisdev/mobassir/data/tts/bn_tts/male/mono/metadata_male.txt", path=os.path.join("/home/apsisdev/mobassir/data/tts/bn_tts/male/mono/", "")
)


In [6]:
dataset_config

BaseDatasetConfig(formatter='', dataset_name='ljspeech', path='/home/apsisdev/mobassir/data/tts/bn_tts/male/mono/', meta_file_train='/home/apsisdev/mobassir/data/tts/bn_tts/male/mono/metadata_male.txt', ignored_speakers=None, language='', meta_file_val='', meta_file_attn_mask='')

In [7]:
# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config,formatter=formatter, eval_split=True)

 | > Found 6187 files in /home/apsisdev/mobassir/data/tts/bn_tts/male/mono


In [8]:
print(len(train_samples),len(eval_samples))

6126 61


In [9]:
train_samples[0]

{'text': 'সকাল সাড়ে নটায় লিলুয়া রেল ওয়ার্কশপে,পরিত্যক্ত পাটের স্তুপে,আগুন লাগে\n',
 'audio_file': '/home/apsisdev/mobassir/data/tts/bn_tts/male/mono/wav/train_bengalimale_01658.wav',
 'speaker_name': 'ljspeech',
 'root_path': '/home/apsisdev/mobassir/data/tts/bn_tts/male/mono/',
 'language': '',
 'audio_unique_name': 'ljspeech#wav/train_bengalimale_01658'}

In [10]:
# we use the same path as this script as our training folder.
output_path = '/home/apsisdev/mobassir/data/tts/'

audio_config = BaseAudioConfig(
     sample_rate = 22050,
    resample =False
)

if(male):
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "তট৫ভিঐঋখঊড়ইজমএেঘঙসীঢ়হঞ‘ঈকণ৬ঁৗশঢঠ\u200c১্২৮দৃঔগও—ছউংবৈঝাযফ\u200dচরষঅৌৎথড়৪ধ০ুূ৩আঃপয়’নলো",
    punctuations = "-!,|.? ",
    )
else:
    characters_config = CharactersConfig(
    pad = '<PAD>',
    eos = '।', #'<EOS>', #'।',
    bos = '<BOS>',# None,
    blank = '<BLNK>',
    phonemes = None,
    characters =  "ইগং়’ুঃন১ঝূও‘ঊোছপফৈ৮ষযৎঢঈকঠিজ০৬ীটডএঅঋধচে২৩ণউয়ঢ়খলভৗসহ্ড়দথবঔাঞশরৌম—ঐআৃঘঙ\u200cঁ৪৫ত",
    punctuations = ".?-!|, ",
    )

# GlowTTSConfig: all model related values for training, validating and testing.

config = GlowTTSConfig(
    #use_speaker_embedding = True,
    run_name="bn_glowtts_male",
    batch_size=200,
    eval_batch_size=128,
    num_loader_workers=16,
    num_eval_loader_workers=16,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=5000,
    text_cleaner="collapse_whitespace",
    use_phonemes=False,
#     phoneme_language="bn",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=50,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    audio = audio_config,
    characters = characters_config,
    cudnn_benchmark=True,
    save_step=1000,
    test_sentences = [
#         "পিপলস ইন্স্যুরেন্স অব চায়না ছেষট্টি বছর আগে ব্যবসা চালু করে।",
#         "সোনার বাংলা আমি তোমায় ভালবাসি।"
        "হয়,হয়ে,ওয়া,হয়েছ,হয়েছে,দিয়ে,যায়,দায়,নিশ্চয়,আয়,ভয়,নয়,আয়াত,নিয়ে,হয়েছে,দিয়েছ,রয়ে,রয়েছ,রয়েছে।",
        "দেয়,দেওয়া,বিষয়,হয়,হওয়া,সম্প্রদায়,সময়,হয়েছি,দিয়েছি,হয়,হয়েছিল,বিষয়ে,নয়,কিয়াম,ইয়া,দেয়া,দিয়েছে,আয়াতে,দয়া।",
        "হওয়ার,হয়েছে,নিশ্চয়ই,রায়,কিয়ামত,উভয়,দিয়েছেন,দুনিয়া,ন্যায়,অবস্থায়,যায়,ফিরিয়ে,দিয়েছিল,ভয়ে,দ্বিতীয়,দায়ক,পায়।",
        "গিয়ে,চেয়ে,হিদায়াত,দায়ে,নিয়েছ,রয়েছে,শয়তান,কিয়ামতে,সম্প্রদায়ে,সম্প্রদায়ের,নেয়,জয়,কিয়ামতের,স্থায়ী,যাওয়া,দয়ালু।",
        "ইয়াহুদ,নয়,ব্যয়,ইয়াহুদী,নেওয়া,উভয়ে,যায়,হয়েছিল,প্রয়োজন।"
    ]
)

In [11]:

ap = AudioProcessor.init_from_config(config)


 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [12]:
ap.resample

False

In [13]:

tokenizer, config = TTSTokenizer.init_from_config(config)

In [14]:

model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [15]:
model

GlowTTS(
  (encoder): Encoder(
    (emb): Embedding(87, 192)
    (prenet): ResidualConv1dLayerNormBlock(
      (conv_layers): ModuleList(
        (0): Conv1d(192, 192, kernel_size=(5,), stride=(1,), padding=(2,))
        (1): Conv1d(192, 192, kernel_size=(5,), stride=(1,), padding=(2,))
        (2): Conv1d(192, 192, kernel_size=(5,), stride=(1,), padding=(2,))
      )
      (norm_layers): ModuleList(
        (0): LayerNorm()
        (1): LayerNorm()
        (2): LayerNorm()
      )
      (proj): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
    )
    (encoder): RelativePositionTransformer(
      (dropout): Dropout(p=0.1, inplace=False)
      (attn_layers): ModuleList(
        (0): RelativePositionMultiHeadAttention(
          (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))


In [16]:
if(is_pretrain):
    pretrain_path =  '/home/apsisdev/mobassir/data/tts/bn_glowtts_male/'
else:
    pretrain_path = ''

In [17]:
# using continue_path argument below for retraining model from last saved weight https://github.com/coqui-ai/Trainer/blob/main/trainer/trainer.py#L210

trainer = Trainer(
    TrainerArgs(continue_path = pretrain_path), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 20
 | > Num. of Torch Threads: 10
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: True

 > Model has 28601809 parameters


In [ ]:
%%time

# AND... 3,2,1... 🚀
trainer.fit()


 > EPOCH: 0/5000
 --> /home/apsisdev/mobassir/data/tts/bn_glowtts_male-October-31-2022_08+42PM-0000000




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 6126



 > TRAINING (2022-10-31 20:42:38) 


 | > Preprocessing samples
 | > Max text length: 114
 | > Min text length: 16
 | > Avg text length: 64.72233104799217
 | 
 | > Max audio length: 276757.0
 | > Min audio length: 49474.0
 | > Avg audio length: 129107.80362389814
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



   --> STEP: 0/31 -- GLOBAL_STEP: 0
     | > current_lr: 0.00000 
     | > step_time: 1.82090  (1.82094)
     | > loader_time: 18.03360  (18.03360)


 > EVALUATION 





> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 61
 | > Preprocessing samples
 | > Max text length: 92
 | > Min text length: 40
 | > Avg text length: 64.52459016393442
 | 
 | > Max audio length: 171374.0
 | > Min audio length: 68524.0
 | > Avg audio length: 126232.5081967213
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 1.28097 (+0.00000)
     | > avg_loss: 3.93224 (+0.00000)
     | > avg_log_mle: 0.75578 (+0.00000)
     | > avg_loss_dur: 3.17645 (+0.00000)

 > BEST MODEL : /home/apsisdev/mobassir/data/tts/bn_glowtts_male-October-31-2022_08+42PM-0000000/best_model_31.pth

 > EPOCH: 1/5000
 --> /home/apsisdev/mobassir/data/tts/bn_glowtts_male-October-31-2022_08+42PM-0000000

 > TRAINING (2022-10-31 20:44:34) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 6126
 | > Preprocessing samples
 | > Max text length: 114
 | > Min text length: 16
 | > Avg text length: 64.72233104799217
 | 
 | > Max audio length: 276757.0
 | > Min audio length: 49474.0
 | > Avg audio length: 129107.80362389814
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


 ! Run is kept in /home/apsisdev/mobassir/data/tts/bn_glowtts_male-October-31-2022_08+42PM-0000000


In [ ]:
!nvidia-smi

In [ ]:
np.__version__#1.21.6